<a href="https://colab.research.google.com/github/EyalKoubi/CloudTur/blob/main/tur5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import IPython

import ipywidgets as widgets
from IPython.display import display
from firebase import firebase
from enum import Enum

FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

quiz = ['', '', '', '', '', '']

error_label = ''

class Quiz(Enum):
  QUESTION = 0
  OP1 = 1
  OP2 = 2
  OP3 = 3
  OP4 = 4
  ANS = 5

submit_button = ''
flag = True

def addItem(question, options, answer):
  question_block = {
      'question': question,
      'options': options,
      'answer': answer,
  }
  result = FBconn.post('/myQuestions/', question_block)

def addItems():

  def on_exit_button_click(button):

    global quiz
    global submit_button

    for i in range(len(quiz)):
      quiz[i].close()

    submit_button.close()
    button.close()

  def showInsertion():

      global flag
      global quiz
      global submit_button
      global Quiz
      global exit_button

      quiz[Quiz.QUESTION.value] = widgets.Text(
          value='',
          placeholder='Enter question',
          description='Question:',
          disabled=False
      )

      quiz[Quiz.OP1.value] = widgets.Text(
          value='',
          placeholder='Enter option 1',
          description='Option 1:',
          disabled=False
      )

      quiz[Quiz.OP2.value] = widgets.Text(
          value='',
          placeholder='Enter option 2',
          description='Option 2:',
          disabled=False
      )

      quiz[Quiz.OP3.value] = widgets.Text(
          value='',
          placeholder='Enter option 3',
          description='Option 3:',
          disabled=False
      )
      
      quiz[Quiz.OP4.value] = widgets.Text(
          value='',
          placeholder='Enter option 4',
          description='Option 4:',
          disabled=False
      )

      quiz[Quiz.ANS.value] = widgets.Text(
          value='',
          placeholder='Enter the right answer',
          description='right answer:',
          disabled=False
      )

      for i in range(len(quiz)):
        display(quiz[i])

      if (flag):
          submit_button = widgets.Button(description="Submit")
          exit_button = widgets.Button(description="Exit")
          hbox = widgets.HBox([submit_button, exit_button])
          display(hbox)

      submit_button.on_click(on_submit_button_click)
      exit_button.on_click(on_exit_button_click)

  def on_submit_button_click(button):

    global exit_button
    global error_label

    question = quiz[Quiz.QUESTION.value].value
    options = [quiz[Quiz.OP1.value].value, quiz[Quiz.OP2.value].value, quiz[Quiz.OP3.value].value, quiz[Quiz.OP4.value].value]
    answer = quiz[Quiz.ANS.value].value

    if (isinstance(error_label, widgets.Widget)):
        error_label.close()

    if not all([question, options[0], options[1], options[2], options[3], answer]):
        error_label = widgets.Label(value="Please fill all the fields!")
        return display(error_label)

    if not (answer in options):
      error_label = widgets.Label(value="The right answer is not in the option!")
      return display(error_label)

    for i in range(len(quiz)):
      quiz[i].close()

    submit_button.close()
    exit_button.close()

    addItem(question, options, answer)
    showInsertion()

  showInsertion()

addItems()

Text(value='', description='Question:', placeholder='Enter question')

Text(value='', description='Option 1:', placeholder='Enter option 1')

Text(value='', description='Option 2:', placeholder='Enter option 2')

Text(value='', description='Option 3:', placeholder='Enter option 3')

Text(value='', description='Option 4:', placeholder='Enter option 4')

...

Text(value='', description='Option 1:', placeholder='Enter option 1')

Text(value='', description='Option 2:', placeholder='Enter option 2')

Text(value='', description='Option 3:', placeholder='Enter option 3')

Text(value='', description='Option 4:', placeholder='Enter option 4')

Text(value='', description='right answer:', placeholder='Enter the right answer')

In [80]:
import ipywidgets as widgets
import time
from IPython.display import display
from firebase import firebase
from enum import Enum
import random

FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

questions = []

question_label = ''
radio_buttons = ''
submit_button = ''
result_label = ''

class Quiz(Enum):
  QUESTION = 0
  OP1 = 1
  OP2 = 2
  OP3 = 3
  OP4 = 4
  ANS = 5

class QuizQuestion:
    def __init__(self, question, options, answer):
        self.question = question
        self.options = options
        self.answer = answer

def get_random_question():
    global questions
    if len(questions) == 0:
        return None
    else:
        index = random.randint(0, len(questions) - 1)
        question = questions[index]
        del questions[index]
        return question

def retrieveQuestions():
    global questions
    questions = []

    # Retrieve all questions from Firebase
    result = FBconn.get('/myQuestions/', None)

    for key, val in result.items():
        question = val['question']
        options = val['options']
        answer = val['answer']
        questions.append(QuizQuestion(question, options, answer))


def displayQuiz():
    retrieveQuestions()
    global questions
    question = get_random_question()
    correct_answers = 0
    total_questions = len(questions)
    
    def checkAnswer(answer, correct_answer):

        global result_label

        nonlocal correct_answers
        if answer == correct_answer:
            correct_answers += 1
            result_label.value = "Correct!"
        else:
            result_label.value = f"Incorrect. The correct answer is {correct_answer}."
    
    def on_submit_button_clicked(b):

        global question_label
        global radio_buttons
        global result_label
        global submit_button

        checkAnswer(radio_buttons.value, question.answer)
        time.sleep(2)

        question_label.close()
        radio_buttons.close()
        submit_button.close()
        result_label.close()
        submit_button.disabled = True
        displayQuiz()

    
    if question is not None:

        global question_label
        global radio_buttons
        global result_label
        global submit_button

        # Display the question
        question_label = widgets.Label(f"{question.question}")
        display(question_label)
        
        # Display the answer choices
        radio_buttons = widgets.RadioButtons(options=question.options, layout={'width': 'max-content'})
        display(radio_buttons)
        
        # Create a submit button
        submit_button = widgets.Button(description="Submit")
        display(submit_button)
        
        # Create a label to display the result
        result_label = widgets.Label(value="")
        display(result_label)
        
        submit_button.on_click(on_submit_button_clicked)

displayQuiz()


Label(value='ts?')

RadioButtons(layout=Layout(width='max-content'), options=('dsfd', 'dsfsdf', 'e', 'dsfds'), value='dsfd')

Button(description='Submit', style=ButtonStyle())

Label(value='')

Label(value='re?')

...

Label(value='')

Label(value='rfd?')

RadioButtons(layout=Layout(width='max-content'), options=('sadasd', 'i', 'sadsa', 'asdas'), value='sadasd')

Button(description='Submit', style=ButtonStyle())

Label(value='')